In [4]:
import re
import pandas as pd
from openpyxl import load_workbook
pd.options.display.max_rows = None

In [59]:
# 開啟檔案
def open_brd_file(filepath):
    f = open(filepath)
    return f

# parse net name
def parse_net_name(string):
    # 定義正規表達式
    netNameRex = re.compile(r"^\s-?[A-Z]+[0-9]*_?[A-Z]+[0-9]*.*")
    netName = netNameRex.findall(string)
    return netName

# 找到總長度的欄位
def total(string):
    # 定義正規表達式
    totalRex = re.compile("TOTAL")
    total = totalRex.findall(string)
    return total
#找到位置
def parse_net(string):
#     locationRex = re.compile(r"([U|R|C|L|J]\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
#     locationRex = re.compile(r"([U|R|C|L|J][A-Z]+\d+\.[A-Z]*\d*|[U|R|C|L|J][A-Z]+\.[A-Z]*\d*|[U|R|C|L|J]\d+.[A-Z]*\d*|[U|R|C|L|J][A-Z]*\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
    locationRex = re.compile(r"([U|R|C|L|J|T|P][A-Z]+\d+\.[A-Z]*\d*|[U|R|C|L|J|T|P][A-Z]+\.[A-Z]*\d*|[U|R|C|L|J|T|P|Q]\d+\.[A-Z]*\d*|[U|R|C|L|J|T|P][A-Z]*\d+\.[A-Z]*\d*|VIA\(T\)|VIA\s)")
    location = locationRex.findall(string)
    if location and location[0] == "VIA ":
        location[0] = location[0].replace(" ", "")
    return location

#找到單一段長度
def sub_length(string):
    lengthRex = re.compile(r"(\d+\.\d+)")
    length = lengthRex.findall(string)
    if length:
        length = length[-1]
    return length 

# 找到層數
def layer_num(string):
    layerRex = re.compile(r"BOTTOM|TOP|L\d+$|IN\d+$")
    layer = layerRex.findall(string)
    return layer

#找到總長度
def total_length(string):
    ttlLength = re.findall(r"\d+.\d+", string)[0]
    return ttlLength

#存csv
def save_csv(datalist, filename):
    df = pd.DataFrame(datalist)
    df.to_csv(f'{filename}.csv', index = False)
    
# 存excel
def save_excel(write, df, sheetName):
#     df = pd.DataFrame(datalist)
    df.to_excel(write, sheet_name=f'{sheetName}', index=False)
    
#刪多餘的raw
def deleteNullVIAROW(df):
    for i in range(df.shape[0]):
        if df.loc[i, "location"] == "VIA" and df.loc[i, "gap"] == 0.0:
            df.drop(i, axis=0, inplace=True)
            
#這一份txt total的path 數量
def branchPathNum(df):
    #扣掉net name/start_end/total length
    maxPathNum = int((df.shape[1] - 7) / 2)
    return maxPathNum

In [67]:
#step1: 開啟檔案
filepath = r"0424.txt"
f = open_brd_file(filepath)
print(f"step1: {filepath} open.")

#step2: 解析檔案
SQS = []
summary = []
netNameList = []

for i in f:
    i = i.replace("\n","")#去掉換行符號
    #如果不是net name 也不是 total
    if not parse_net_name(i):
        if not parse_net(i):
            if not total(i):
                continue
            
    #如果是net name
    if parse_net_name(i):
        data = {} #清空
        netName = i.replace(" ", "")
        #建立netnamelist
        netNameList.append(netName)
        netPath = ""
#         data.update({"net_name" : netName})
#         SQS.append(data)
#         print(data)
    
    #如果是location
#     elif parse_net(i):
#         data = {} #清空
        
#         if sub_length(i):
#             location = parse_net(i)[0]
#             length = float(sub_length(i))
#             layer = layer_num(i)[0] if layer_num(i) else ""
#             data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
# #         print(data)
#             SQS.append(data)
    elif parse_net(i):
        data = {} #清空            
        if sub_length(i):
            location = parse_net(i)[0]
            
            length = float(sub_length(i))
            layer = layer_num(i)[0] if layer_num(i) else ""
            data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
            SQS.append(data)
        
    
    #如果是total那一行
    elif total(i):
        data = {} #清空
        ttlLength = float(total_length(i))
        data.update({"net_name" : netName, "total_length" : ttlLength})
#         print(data)
        summary.append(data)
        
print(f"step2: {filepath} parsed done.")


#存取df
dfsummary, dfSQS, dfSQSR = pd.DataFrame(summary), pd.DataFrame(SQS), pd.DataFrame(SQS)

#修改df
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    
    # 填第一個元件的layer空格 
    # ex. TOP = TOP
    dfSQSR.loc[indexList[0],"layer"] = dfSQSR.loc[indexList[1],"layer"]
    
    #每個netname裡面找gap
    length = len(indexList)
    for index in indexList[::-1]:
        if length > 1:
            gap = dfSQSR.loc[index]["length"] - dfSQSR.loc[index-1]["length"]
            dfSQSR.loc[index,"gap"] = gap
        else:
            gap = 0
            dfSQSR.loc[index,"gap"] = gap
        length -= 1
#         print(index, gap)

#刪除多餘的VIA
deleteNullVIAROW(dfSQSR)    
    
#在summary建立branch path & branch length
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    path = ""
    connIdxList = []
    MS = 0
    SL = 0
    NAN = 0
    
    for idx in indexList:
        
        ####################################################  前面的表層裡層         
        if re.findall("BOTTOM|TOP", dfSQSR.loc[idx, "layer"]): 
            MS += dfSQSR.loc[idx, "gap"]
        elif re.findall("L\d+$|IN\d+$", dfSQSR.loc[idx, "layer"]):
            SL += dfSQSR.loc[idx, "gap"]
        else:
            NAN += dfSQSR.loc[idx, "gap"]
        
        ##########################################################
        if not re.findall("VIA", dfSQSR.loc[idx, "location"]): #找出connnector index 排出VIA, VIA(T)
            connIdxList.append(idx)
    
    length = len(connIdxList)
    
    #找出connector 在每條indexlist 裡面的index
    num = 0
    netIndex = dfsummary[dfsummary["net_name"] == f"{i}"].index.tolist()
    
    #把起始點&終點 加入summary
    start_end = f"{dfSQSR.loc[indexList[0], 'location']}:{dfSQSR.loc[indexList[-1], 'location']}"

    dfsummary.loc[netIndex, "start_end_path"] = start_end
    
    print(MS, SL)
    ####################在這裡加入表層裡層
    dfsummary.loc[netIndex, "path_MS"] = start_end + "-MS"
    dfsummary.loc[netIndex, "length_MS"] = MS
    dfsummary.loc[netIndex, "path_SL"] = start_end + "-SL"
    dfsummary.loc[netIndex, "length_SL"] = SL
    
    
    for idx in range(length - 1):
        num +=1 
        pathIdx = []
        pathIdx.append(indexList.index(connIdxList[idx]))
        pathIdx.append(indexList.index(connIdxList[idx + 1]))
               
        #抓出兩兩conn
        connector = ""
        for j in pathIdx:
            connector += dfSQSR.loc[indexList[j], "location"] + ":"
        
        connector = connector[:-1] 
        dfsummary.loc[netIndex, f"path{num}"] = connector
        
        #算出兩兩conn間的長度
        branchLen = 0
        for k in indexList[ pathIdx[0]+1 : pathIdx[1]+1]:
            branchLen += dfSQSR.loc[k, "gap"]

        dfsummary.loc[netIndex, f"length{num}"] = branchLen

        
#調整dfsummary的順序
startEndColumn = dfsummary.pop(dfsummary.columns[2])
dfsummary.insert(1, startEndColumn.name, startEndColumn)
   

#轉成final SQS資料
column1 = []
column2 = []
for row in range(dfsummary.shape[0]):
    column1.append(dfsummary.loc[row,"net_name"])
    column1.append(dfsummary.loc[row,"start_end_path"])
    column1.append(dfsummary.loc[row,"path_MS"])
    column1.append(dfsummary.loc[row,"path_SL"])
    
    column2.append("")
    column2.append(dfsummary.loc[row,"total_length"])
    column2.append(dfsummary.loc[row,"length_MS"])
    column2.append(dfsummary.loc[row,"length_SL"])
    


    for num in range(branchPathNum(dfsummary)):

        column1.append(dfsummary.loc[row,f"path{num + 1}"])
        column2.append(dfsummary.loc[row,f"length{num + 1}"])
        
final = {
    "SQS"    : column1,
    "length" :column2
}
#轉成df 順便去掉空值
dfFinal = pd.DataFrame(final).dropna(axis=0)        
    
    
# step3: 儲存檔案
filename = re.findall(r"^\w*", filepath)[0]
write = pd.ExcelWriter(f'{filename}.xlsx')
save_excel(write, dfFinal, "final")
save_excel(write, dfsummary, "summary")
save_excel(write, dfSQS, "data base")
save_excel(write, dfSQSR, "SQS")
write.save()

print(f"step3: {filename}.xlsx saved.")

step1: 0424.txt open.
step2: 0424.txt parsed done.
44.986999999999895 1275.092
44.80799999999992 1061.759
104.93299999999985 1052.5690000000002
80.58000000000004 943.8149999999999
237.117 1137.392
232.06799999999996 985.06
200.7410000000001 1339.6899999999998
263.499 1481.847
45.49299999999998 1365.496
112.84300000000003 794.049
48.03400000000008 1178.2749999999999
43.49300000000004 960.05
114.51600000000003 1153.009
189.5330000000001 1031.088
80.40800000000013 1065.186
43.389000000000024 1434.176
45.45199999999985 1209.9360000000001
115.94400000000005 903.156
186.70300000000006 1136.49
45.06 928.1500000000001
130.59900000000005 1287.5230000000001
44.64099999999992 985.005
120.16499999999992 1036.944
91.19000000000004 881.4159999999999
140.12 1294.6660000000002
44.68800000000003 997.7520000000001
43.92499999999997 1342.73
44.976000000000035 898.427
198.95499999999987 1339.509
279.12300000000005 1472.989
94.92 0
187.10500000000002 0
44.67799999999992 934.9830000000001
44.98100000000004 

In [62]:
branchPathNum(dfsummary)

6

In [60]:
dfsummary

,net_name,start_end_path,total_length,path_MS,length_MS,path_SL,length_SL,path1,length1,path2,length2,path3,length3,path4,length4,path5,length5,path6,length6
0,-FBA_ABI_H,UGPU1.K22:UV2.J4,1320.079,UGPU1.K22:UV2.J4-MS,72610.625,UGPU1.K22:UV2.J4-SL,827879.078,UGPU1.K22:UV2.J4,1320.079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-FBA_ABI_L,UGPU1.F23:UV1.J4,1106.567,UGPU1.F23:UV1.J4-MS,72655.433,UGPU1.F23:UV1.J4-SL,828940.837,UGPU1.F23:UV1.J4,1106.567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-FBA_CAS_H,UGPU1.J26:UV2.L3,1157.502,UGPU1.J26:UV2.L3-MS,72760.366,UGPU1.J26:UV2.L3-SL,829993.406,UGPU1.J26:UV2.L3,1157.502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-FBA_CAS_L,UGPU1.G26:UV1.L3,1024.395,UGPU1.G26:UV1.L3-MS,72840.946,UGPU1.G26:UV1.L3-SL,830937.221,UGPU1.G26:UV1.L3,1024.395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-FBA_CKE_H,UGPU1.J27:R4501.2,1374.509,UGPU1.J27:R4501.2-MS,73078.063,UGPU1.J27:R4501.2-SL,832074.613,UGPU1.J27:UV2.J3,1270.342,UV2.J3:R4501.2,104.167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-FBA_CKE_L,UGPU1.G27:R4502.2,1217.128,UGPU1.G27:R4502.2-MS,73310.131,UGPU1.G27:R4502.2-SL,833059.673,UGPU1.G27:UV1.J3,1059.470,UV1.J3:R4502.2,157.658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-FBA_CLK0,UGPU1.D25:R4516.1,1540.431,UGPU1.D25:R4516.1-MS,73510.872,UGPU1.D25:R4516.1-SL,834399.363,UGPU1.D25:UV1.J11,1384.935,UV1.J11:R4513.1,120.496,R4513.1:R4516.1,35.000,NaN,NaN,NaN,NaN,NaN,NaN
7,-FBA_CLK1,UGPU1.M22:R4518.1,1745.346,UGPU1.M22:R4518.1-MS,73774.371,UGPU1.M22:R4518.1-SL,835881.210,UGPU1.M22:UV2.J11,1527.230,UV2.J11:R4515.1,182.702,R4515.1:R4518.1,35.414,NaN,NaN,NaN,NaN,NaN,NaN
8,-FBA_CS_H,UGPU1.M24:UV2.G12,1410.989,UGPU1.M24:UV2.G12-MS,73819.864,UGPU1.M24:UV2.G12-SL,837246.706,UGPU1.M24:UV2.G12,1410.989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-FBA_CS_L,UGPU1.C27:UV1.G12,906.892,UGPU1.C27:UV1.G12-MS,73932.707,UGPU1.C27:UV1.G12-SL,838040.755,UGPU1.C27:UV1.G12,906.892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
dfsummary.shape

(127, 19)

In [9]:
df = pd.DataFrame(dfSQS)
df1 = pd.DataFrame(dfSQS)
df2 = pd.DataFrame(dfSQS)

In [30]:
df1 = dfSQSR
df2 = dfSQSR
dfsummarytest = dfsummary
df1.shape

(523, 5)

In [42]:
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
#     print(indexList)
        
    print(i)
    MS = 0
    SL = 0
    NAN = 0
    for idx in indexList:
        
        if re.findall("BOTTOM|TOP", dfSQSR.loc[idx, "layer"]): 
            MS += dfSQSR.loc[idx, "gap"]
        elif re.findall("L\d+$|IN\d+$", dfSQSR.loc[idx, "layer"]):
            SL += dfSQSR.loc[idx, "gap"]
        else:
            NAN += dfSQSR.loc[idx, "gap"]
        
    print("MS:", MS)
    print("SL:", SL)
    print("NAN:", NAN)
    print("-"*100)

-FBA_ABI_H
MS: 44.986999999999895
SL: 1275.092
NAN: 0
----------------------------------------------------------------------------------------------------
-FBA_ABI_L
MS: 44.80799999999992
SL: 1061.759
NAN: 0
----------------------------------------------------------------------------------------------------
-FBA_CAS_H
MS: 104.93299999999985
SL: 1052.5690000000002
NAN: 0
----------------------------------------------------------------------------------------------------
-FBA_CAS_L
MS: 80.58000000000004
SL: 943.8149999999999
NAN: 0
----------------------------------------------------------------------------------------------------
-FBA_CKE_H
MS: 237.117
SL: 1137.392
NAN: 0
----------------------------------------------------------------------------------------------------
-FBA_CKE_L
MS: 232.06799999999996
SL: 985.06
NAN: 0
----------------------------------------------------------------------------------------------------
-FBA_CLK0
MS: 200.7410000000001
SL: 1339.6899999999998
NAN: 0
----

NAN: 0
----------------------------------------------------------------------------------------------------
FBA_EDC3
MS: 88.81700000000002
SL: 1245.762
NAN: 0
----------------------------------------------------------------------------------------------------
FBA_EDC4
MS: 46.45999999999985
SL: 1106.3120000000001
NAN: 0
----------------------------------------------------------------------------------------------------
FBA_EDC5
MS: 49.44000000000001
SL: 1479.127
NAN: 0
----------------------------------------------------------------------------------------------------
FBA_EDC6
MS: 226.94799999999998
SL: 562.139
NAN: 0
----------------------------------------------------------------------------------------------------
FBA_EDC7
MS: 86.09500000000008
SL: 632.279
NAN: 0
----------------------------------------------------------------------------------------------------
FBA_MA10_H
MS: 45.10300000000008
SL: 1202.114
NAN: 0
----------------------------------------------------------------------

In [41]:
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
#     print(indexList)
        
    print(i)
    MS = 0
    SL = 0
    for idx in indexList:
        
        if re.findall("BOTTOM|TOP", dfSQSR.loc[idx, "layer"]): #找出connnector index 排出VIA, VIA(T)
#             connIdxList.append(idx)

            print(dfSQSR.loc[idx, "layer"])
            print(dfSQSR.loc[idx, "gap"])
            MS += dfSQSR.loc[idx, "gap"]
        elif re.findall("L\d+$|IN\d+$", dfSQSR.loc[idx, "layer"]):
            print(dfSQSR.loc[idx, "layer"])
            print(dfSQSR.loc[idx, "gap"])
            SL += dfSQSR.loc[idx, "gap"]
        else:
            print("!!!!!!!!!!!!!!!!!!!!:", dfSQSR.loc[idx, "layer"])
        
    print("MS:", MS)
    print("SL:", SL)
    print("-"*100)

-FBA_ABI_H
BOTTOM
0.0
BOTTOM
22.473
IN3
1275.092
BOTTOM
22.513999999999896
MS: 44.986999999999895
SL: 1275.092
----------------------------------------------------------------------------------------------------
-FBA_ABI_L
BOTTOM
0.0
BOTTOM
22.276
IN3
1061.759
BOTTOM
22.531999999999925
MS: 44.80799999999992
SL: 1061.759
----------------------------------------------------------------------------------------------------
-FBA_CAS_H
BOTTOM
0.0
BOTTOM
82.542
IN3
1052.5690000000002
BOTTOM
22.39099999999985
MS: 104.93299999999985
SL: 1052.5690000000002
----------------------------------------------------------------------------------------------------
-FBA_CAS_L
BOTTOM
0.0
BOTTOM
58.168
IN3
943.8149999999999
BOTTOM
22.412000000000035
MS: 80.58000000000004
SL: 943.8149999999999
----------------------------------------------------------------------------------------------------
-FBA_CKE_H
BOTTOM
0.0
BOTTOM
110.559
IN3
1137.392
BOTTOM
22.391000000000076
TOP
104.16699999999992
MS: 237.117
SL: 11

SL: 610.4359999999999
----------------------------------------------------------------------------------------------------
FBA_D62
BOTTOM
0.0
BOTTOM
44.783
IN4
503.254
BOTTOM
22.390999999999963
MS: 67.17399999999996
SL: 503.254
----------------------------------------------------------------------------------------------------
FBA_D63
BOTTOM
0.0
BOTTOM
25.949
IN4
616.2520000000001
BOTTOM
22.390999999999963
MS: 48.33999999999996
SL: 616.2520000000001
----------------------------------------------------------------------------------------------------
FBA_DBI0
BOTTOM
0.0
BOTTOM
23.732
IN1
919.8340000000001
BOTTOM
22.41199999999992
MS: 46.14399999999992
SL: 919.8340000000001
----------------------------------------------------------------------------------------------------
FBA_DBI1
BOTTOM
0.0
BOTTOM
23.627
IN4
1301.3210000000001
BOTTOM
22.421999999999798
MS: 46.04899999999979
SL: 1301.3210000000001
-------------------------------------------------------------------------------------------

In [32]:
 dfSQSR 

,net_name,location,length,layer,gap
0,-FBA_ABI_H,UGPU1.K22,0.000,BOTTOM,0.000
1,-FBA_ABI_H,VIA,22.473,BOTTOM,22.473
2,-FBA_ABI_H,VIA,1297.565,IN3,1275.092
3,-FBA_ABI_H,UV2.J4,1320.079,BOTTOM,22.514
4,-FBA_ABI_L,UGPU1.F23,0.000,BOTTOM,0.000
5,-FBA_ABI_L,VIA,22.276,BOTTOM,22.276
6,-FBA_ABI_L,VIA,1084.035,IN3,1061.759
7,-FBA_ABI_L,UV1.J4,1106.567,BOTTOM,22.532
8,-FBA_CAS_H,UGPU1.J26,0.000,BOTTOM,0.000
9,-FBA_CAS_H,VIA,82.542,BOTTOM,82.542


In [ ]:
df1.loc[323]["length"]
df1.loc[322]["length"]

In [ ]:
df1.loc[322:323]["length"]

In [ ]:
df1.loc[322:323]["length"].tolist()

In [ ]:
df1.loc["net_name"]= "PCIE12_L2_TXP_C"